In [20]:
import sklearn.feature_extraction.text
import sklearn.metrics.pairwise
import pandas
import numpy
import plotly.express as px
import spacy
import simplemma
import random

In [21]:
model_es = spacy.load("es_core_news_sm")

In [22]:
def reemplazar_tildes(texto: str) -> str:
    reemplazos = {"á": "a", "é": "e", "í": "i", "ó": "o", "ú": "u"}
    for original, reemplazo in reemplazos.items():
        texto = texto.replace(original, reemplazo)
    return texto


def normalize_text(text: str) -> str:
    doc = model_es(" ".join(simplemma.text_lemmatizer(text, lang="es")))
    return " ".join(
        [
            reemplazar_tildes(str(token))
            for token in doc
            if token.is_alpha and not token.is_stop
        ]
    )

In [23]:
df_normalize = pandas.read_csv("./data_normalize.csv")
df_normalize

,Genero,Artista,Titulo,Cancion,normalize
0,Pop,Shakira,Waka Waka (Esto Es África),"Llegó el momento, caen las murallas\r\nVa a co...",pop pop pop pop pop shakira shakira shakira sh...
1,Pop,Shakira,Gitana,Nunca usé un antifaz\r\nVoy de paso por este m...,pop pop pop pop pop shakira shakira shakira sh...
2,Pop,Shakira,"Te Aviso, Te Anuncio (Tango)",Nunca pensé que doliera el amor así\r\nCuando ...,pop pop pop pop pop shakira shakira shakira sh...
3,Pop,Shakira,Addicted To You,Debe ser el perfume que usas\r\nO el agua con ...,pop pop pop pop pop shakira shakira shakira sh...
4,Pop,Shakira,Monotonía (part. Ozuna),"No fue culpa tuya, ni tampoco mía\r\nFue culpa...",pop pop pop pop pop shakira shakira shakira sh...
...,...,...,...,...,...
3060,Salsa,Willie Gonzalez,No Es Casualidad,Siempre cada encuentro parece el primero\r\nSe...,salsa salsa salsa salsa salsa willie gonzalez ...
3061,Salsa,Willie Gonzalez,No Podras Escapar de Mi,Poder tocar tu mano\r\nEstar siempre a tu lado...,salsa salsa salsa salsa salsa willie gonzalez ...
3062,Salsa,Willie Gonzalez,Doble Vida,Juntos viven tu y el\r\nY estas tan sola...\r\...,salsa salsa salsa salsa salsa willie gonzalez ...
3063,Salsa,Willie Gonzalez,Tan Solo,Me has vuelto a llamar\r\nY como un niño emoci...,salsa salsa salsa salsa salsa willie gonzalez ...


In [24]:
corpus = df_normalize["normalize"].to_list()

In [25]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [26]:
def process_query(query: str) -> str:
    return normalize_text(query)


def get_query_vector(processed_query: str) -> numpy.ndarray:
    query_vector = vectorizer.transform([processed_query]).toarray()
    return query_vector


def get_song_recommendations(query_vector: numpy.ndarray, top_n=5) -> pandas.DataFrame:
    cosine_similarities = sklearn.metrics.pairwise.cosine_similarity(
        query_vector, X
    ).flatten()
    related_docs_indices = cosine_similarities.argsort()[: -top_n - 1 : -1]
    return df_normalize.iloc[related_docs_indices]


def search_and_recommend(query: str, top_n: int = 5) -> pandas.DataFrame:
    processed_query = process_query(query)
    query_vector = get_query_vector(processed_query)
    recommendations = get_song_recommendations(query_vector, top_n)
    return recommendations

In [27]:
top_n = 5

### Prueba

In [55]:
query = "ciega sordomuda"
search_and_recommend(query)

,Genero,Artista,Titulo,Cancion,normalize
10,Pop,Shakira,"Ciega, Sordomuda",Se me acaba el argumento\r\nY la metodología\r...,pop pop pop pop pop shakira shakira shakira sh...
2990,Salsa,Cheo Feliciano,El Ciego,Has visto como pierde su alegría\r\nuna fuente...,salsa salsa salsa salsa salsa cheo feliciano c...
2468,Salsa,Marc Anthony,Muy Dentro De Mi,Solo quiero que vuelvas a mí\r\npara otra vez ...,salsa salsa salsa salsa salsa marc anthony mar...
869,Reggaeton,Blessd,FUMANDO (part. Ryan Castro),"Y si la ven por ahí, yo la extraño a morir\r\n...",reggaeton reggaeton reggaeton reggaeton reggae...
1714,rock,platero y tu,si tú te vas,"Nena, hasta aquí hemos llegado\r\nPuedes hacer...",rock rock rock rock rock platero platero plate...


In [ ]:
df_normalize[df_normalize.Artista == 'Morat']

### Otros Ejemplos

In [28]:
query = "que el padre los proteja que el hijo los bendiga"
df_results = search_and_recommend(query, top_n)
df_results

,Genero,Artista,Titulo,Cancion,normalize
2936,Salsa,Oscar D'Leon,Padre e Hijo,"Oscar: Hijo tenemos que hablar,\r\neducandote,...",salsa salsa salsa salsa salsa oscar d leon osc...
1029,Reggaeton,Farruko,Lo Que Yo Diga (Dema Ga Ge Gi Go Gu Remix) (pa...,Que el padre me los proteja y que el hijo los ...,reggaeton reggaeton reggaeton reggaeton reggae...
1617,rock,maná,bendita tu luz,Bendito el lugar\r\nY el motivo de estar ahí\r...,rock rock rock rock rock mana mana mana mana m...
2742,Salsa,Grupo Niche,Mi hijo y yo,"Juntos, caminemos juntos\r\notra oportunidad q...",salsa salsa salsa salsa salsa grupo niche grup...
2381,Vallenato,Los Betos,Bendita Suerte,Una pareja que con distinto social se llegan a...,vallenato vallenato vallenato vallenato vallen...


In [29]:
query = "bad bunny Antes que este phillie se apague"
df_results = search_and_recommend(query, top_n)
df_results

,Genero,Artista,Titulo,Cancion,normalize
661,Reggaeton,Bad Bunny,Un Coco,"Antes que este phillie se apague, yo te voy a ...",reggaeton reggaeton reggaeton reggaeton reggae...
972,Reggaeton,Arcángel,La Jumpa (part. Bad Bunny) [explícita],"(Ajá, yao)\r\n(Arcángel, pa')\r\n(Once a—)\r\n...",reggaeton reggaeton reggaeton reggaeton reggae...
654,Reggaeton,Bad Bunny,Un Verano Sin Ti,"No sé qué pasó\r\nOtro amor que, de repente, f...",reggaeton reggaeton reggaeton reggaeton reggae...
665,Reggaeton,Bad Bunny,Agosto,"Baby, vamo' a hacerlo bajo las estrella'\r\nY ...",reggaeton reggaeton reggaeton reggaeton reggae...
646,Reggaeton,Bad Bunny,Soy Peor,Salí jodido la última vez que en alguien yo co...,reggaeton reggaeton reggaeton reggaeton reggae...


In [30]:
query = "shakira waka waka"
df_results = search_and_recommend(query, top_n)
df_results

,Genero,Artista,Titulo,Cancion,normalize
0,Pop,Shakira,Waka Waka (Esto Es África),"Llegó el momento, caen las murallas\r\nVa a co...",pop pop pop pop pop shakira shakira shakira sh...
15,Pop,Shakira,Waka Waka (This Time For Africa),You're a good soldier\r\nChoosing your battles...,pop pop pop pop pop shakira shakira shakira sh...
49,Pop,Shakira,Antes de Las Seis,No actúes tan extraño\r\nDuro como una roca\r\...,pop pop pop pop pop shakira shakira shakira sh...
22,Pop,Shakira,Tú,Te regalo mi cintura\r\nY mis labios para cuan...,pop pop pop pop pop shakira shakira shakira sh...
20,Pop,Shakira,Hay Amores,"Ay, mi bien\r\n¿Qué no haría yo por ti?\r\nPor...",pop pop pop pop pop shakira shakira shakira sh...


In [31]:
query = "tus besos frios como la lluvia"
df_results = search_and_recommend(query, top_n)
df_results

,Genero,Artista,Titulo,Cancion,normalize
2780,Salsa,Eddie Santiago,Lluvia,No me digas nada ya lo sabía\r\nQue nuestro ro...,salsa salsa salsa salsa salsa eddie santiago e...
1915,Vallenato,Diomedes Díaz,Lluvia de Verano,Ya no tengo ni penas ni sufrimientos\r\nYa se ...,vallenato vallenato vallenato vallenato vallen...
2401,Vallenato,Los Betos,Lluvia de Mujeres,Hay!!! dices que te marchas lejos y nose porqu...,vallenato vallenato vallenato vallenato vallen...
2748,Salsa,Grupo Niche,Gotas de Lluvia,Gotas de lluvia no es el rocío\r\nLágrimas que...,salsa salsa salsa salsa salsa grupo niche grup...
146,Pop Rock,Morat,11 Besos,Con un beso llegó la calma\r\nCon un beso se f...,pop rock pop rock pop rock pop rock pop rock m...


In [32]:
query = "Héctor, el cantante"
df_results = search_and_recommend(query, top_n)
df_results

,Genero,Artista,Titulo,Cancion,normalize
2648,Salsa,Héctor Lavoe,El Rey de La Puntualidad,EL REY DE LA PUNTUALIDAD\r\nNosotros somos bue...,salsa salsa salsa salsa salsa hector lavoe hec...
2645,Salsa,Héctor Lavoe,El Cantante,Yo soy el cantante\r\nQue hoy han venido a esc...,salsa salsa salsa salsa salsa hector lavoe hec...
2652,Salsa,Héctor Lavoe,Mi Gente,Mi gente\r\n¡Ustedes!\r\nLo más grande de este...,salsa salsa salsa salsa salsa hector lavoe hec...
2661,Salsa,Héctor Lavoe,Loco,"Porque soy así\r\nme llaman loco,\r\nnadie sab...",salsa salsa salsa salsa salsa hector lavoe hec...
2650,Salsa,Héctor Lavoe,Todo Tiene Su Final,Todo tiene su final\r\nNada dura para siempre\...,salsa salsa salsa salsa salsa hector lavoe hec...


In [33]:
query = "que hoy esta mas redonda que los ___ de quiein me nombra"
df_results = search_and_recommend(query, top_n)
df_results

,Genero,Artista,Titulo,Cancion,normalize
1251,rock,albertucho,la persiana,Luna de mala lengua\r\nque hostiga insultos a ...,rock rock rock rock rock albertucho albertucho...
1531,rock,los jaivas,todos juntos,Hace mucho tiempo que yo vivo preguntándome\r\...,rock rock rock rock rock jaivas jaivas jaivas ...
1519,rock,los jaivas,guajira cosmica,El sol le dijo a los hombres\r\nCuiden bien de...,rock rock rock rock rock jaivas jaivas jaivas ...
371,Alternativo,Julieta Venegas,Mujeres,La mujer debe ser bonita\r\nLa mujer debe ser ...,alternativo alternativo alternativo alternativ...
1581,rock,los rodríguez,buena suerte,"Dicen los toreros:\r\n""buena suerte, compañero...",rock rock rock rock rock rodriguez rodriguez r...


In [34]:
query = "marea Quién te ha colgado el mar de las pestañas"
df_results = search_and_recommend(query, top_n)
df_results

,Genero,Artista,Titulo,Cancion,normalize
1676,rock,marea,la luna me sabe a poco,Decía que tenía el corazón alicatao hasta el t...,rock rock rock rock rock marea marea marea mar...
1851,rock,vetusta morla,la marea,"La marea me dejó arenas de plata,\r\nque pondr...",rock rock rock rock rock vetusto morla vetusto...
1684,rock,marea,mierda y cuchara,"Cuéntame, dime, ¿Quién te ha colgado el mar de...",rock rock rock rock rock marea marea marea mar...
1673,rock,marea,el hombre del sacoo,Coro:\r\n(Hay que continuar contra viento y ma...,rock rock rock rock rock marea marea marea mar...
1685,rock,marea,pan duro,Arrugas que son surcos con retoños tiernos\r\n...,rock rock rock rock rock marea marea marea mar...


In [35]:
query = "ruben blades camaleon"
df_results = search_and_recommend(query, top_n)
df_results

,Genero,Artista,Titulo,Cancion,normalize
2532,Salsa,Rubén Blades,Camaleón,"Qué es lo que pasa, camaleón\r\nCalma la envid...",salsa salsa salsa salsa salsa ruben blades rub...
2552,Salsa,Rubén Blades,Todos Vuelven,Todos vuelven a la tierra en que nacieron; al ...,salsa salsa salsa salsa salsa ruben blades rub...
2545,Salsa,Rubén Blades,Cuentas Del Alma,Siempre en la noche mi mamá buscaba el sueño\r...,salsa salsa salsa salsa salsa ruben blades rub...
2556,Salsa,Rubén Blades,Me Recordarás,"Me recordaras, cuando en las tardes muera el s...",salsa salsa salsa salsa salsa ruben blades rub...
2561,Salsa,Rubén Blades,Ya no te puedo querer,No quiero que me supliques\r\nque yo te quiera...,salsa salsa salsa salsa salsa ruben blades rub...


In [36]:
query = "caifanes ya no escucho el corazón"
df_results = search_and_recommend(query, top_n)
df_results

,Genero,Artista,Titulo,Cancion,normalize
1318,rock,caifanes,nada,Siento frío\r\nya no escucho el corazón\r\nen ...,rock rock rock rock rock caifanes caifanes cai...
1311,rock,caifanes,detras de ti,Voy detrás de ti\r\nComo un perro infeliz\r\nV...,rock rock rock rock rock caifanes caifanes cai...
1322,rock,caifanes,para que no digas que no pienso en ti,Ando buscando\r\nLas formas ocultas\r\nDe una ...,rock rock rock rock rock caifanes caifanes cai...
1320,rock,caifanes,no dejes que,Cuando veo a través del vaso\r\nVeo a través d...,rock rock rock rock rock caifanes caifanes cai...
1309,rock,caifanes,ay maría,En algún lugar de un gran país\r\nOlvidaron co...,rock rock rock rock rock caifanes caifanes cai...


In [37]:
query = "diomedes sin medir distancias"
df_results = search_and_recommend(query, top_n)
df_results

,Genero,Artista,Titulo,Cancion,normalize
1873,Vallenato,Diomedes Díaz,Sin Medir Distancias,La herida que siempre llevo en el alma\r\nNo c...,vallenato vallenato vallenato vallenato vallen...
150,Pop,Manuel Medrano,La Distancia,No me acordaba de lo rico que era tocar tu pie...,pop pop pop pop pop manuel medrano manuel medr...
1956,Vallenato,Diomedes Díaz,Que Hubo Linda,Ay yo no sé como pedirte que me quieras\r\nYo ...,vallenato vallenato vallenato vallenato vallen...
1919,Vallenato,Diomedes Díaz,Muchas Gracias,Hoy nuevamente me ha nacido otra canción\r\nPa...,vallenato vallenato vallenato vallenato vallen...
1879,Vallenato,Diomedes Díaz,No Comprendo,Yo no comprendo por qué razón\r\nTe viven dici...,vallenato vallenato vallenato vallenato vallen...


In [19]:
query = "esternocleido nvdoson"
df_results = search_and_recommend(query, top_n)
df_results

,Genero,Artista,Titulo,Cancion,normalize
3064,Salsa,Willie Gonzalez,Si Tú Fueras Mía,"La ilusión, mas grande de toda mi vida\r\nEs t...",salsa salsa salsa salsa salsa willie gonzalez ...
1017,Reggaeton,Farruko,Baby (part. Nicky Jam y Amenazzy),"N-I-C-K\r\nAmenazzy (yeah-ieh)\r\nLa Amenazzy,...",reggaeton reggaeton reggaeton reggaeton reggae...
1026,Reggaeton,Farruko,Su Hija Me Gusta (feat. José Feliciano),"Hola que tal, vengo a decirle un secreto\r\nCo...",reggaeton reggaeton reggaeton reggaeton reggae...
1025,Reggaeton,Farruko,"Liberace (Remix 2) (part. De La Ghetto, Bad Bu...",[De La Ghetto]\r\nMe levanto por la mañana den...,reggaeton reggaeton reggaeton reggaeton reggae...
1024,Reggaeton,Farruko,Pasa_je_ro,Farru\r\n\r\nEn bajita lowkey aprendí yo a viv...,reggaeton reggaeton reggaeton reggaeton reggae...
